In [27]:
from bokeh.io import output_notebook, show, curdoc, push_notebook,output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select
from bokeh.models.widgets import Dropdown
from bokeh.layouts import widgetbox, column, row
from ipywidgets import interact
import pandas as pd
import numpy as np
#output_file("test.html")


In [28]:
path=r'C:\Users\mattk\Documents\Projects\Import_XML\Pickle\Swaption_Results.pkl'
path2=r'C:\Users\mattk\Documents\Projects\Import_XML\Out_csv2\Swaptions_results2.txt'

In [29]:
df = pd.read_pickle(path)
df2 = pd.read_csv(path2,parse_dates=['ValueDate','ExerciseDate'])

In [30]:
df['ValueDate']=pd.to_datetime(df['ValueDate'])
df['ExerciseDate']=pd.to_datetime(df['ExerciseDate'])

In [31]:
df.dtypes

ValueDate                     datetime64[ns]
ClientForward                        float64
ClientImpliedVolatility              float64
ClientPrice                          float64
ConsensusForward                     float64
ImpliedConsensusVolatility           float64
ConsensusPrice                       float64
ExerciseDate                  datetime64[ns]
OptionType                            object
StandardDeviationPrice               float64
StrikeRelative                        object
Tenor                                float64
Exp                                   object
dtype: object

In [32]:
len(df['Tenor'].unique().tolist())

14

In [33]:
df['Tenor'].unique().tolist()

[12.0,
 24.0,
 36.0,
 48.0,
 60.0,
 72.0,
 84.0,
 96.0,
 108.0,
 120.0,
 180.0,
 240.0,
 300.0,
 360.0]

In [34]:
df['Tail'] = df['Tenor']/12

In [35]:
df['Tail']=df['Tail'].astype(str) + 'Y'

In [36]:
tails=list(df['Tail'].unique().tolist())

In [37]:
tails

['1.0Y',
 '2.0Y',
 '3.0Y',
 '4.0Y',
 '5.0Y',
 '6.0Y',
 '7.0Y',
 '8.0Y',
 '9.0Y',
 '10.0Y',
 '15.0Y',
 '20.0Y',
 '25.0Y',
 '30.0Y']

In [41]:
ff=df[
    (df["Exp"]=="1y")&
    (df["Tail"]=='10.0Y')
    ]

In [42]:
ff.head(6)

,ValueDate,ClientForward,ClientImpliedVolatility,ClientPrice,ConsensusForward,ImpliedConsensusVolatility,ConsensusPrice,ExerciseDate,OptionType,StandardDeviationPrice,StrikeRelative,Tenor,Exp,Tail
1974,2017-04-28,2.399824,76.465100,107.402141,2.399268,76.165390,106.602385,2018-04-30,Receiver,0.971794,-50,120.0,1y,10.0Y
1975,2017-04-28,2.399824,75.530907,175.204721,2.399268,75.459544,174.944237,2018-04-30,Receiver,0.748627,-25,120.0,1y,10.0Y
1976,2017-04-28,2.399824,85.087448,12.239091,2.399258,82.247831,10.153507,2018-04-30,Receiver,0.713724,-150,120.0,1y,10.0Y
1977,2017-04-28,2.399824,79.998323,36.999676,2.399269,78.848795,35.088025,2018-04-30,Receiver,0.982367,-100,120.0,1y,10.0Y
1978,2017-04-28,2.399824,75.333460,273.030455,2.399268,75.323350,272.902837,2018-04-30,Receiver,0.495255,ATM,120.0,1y,10.0Y
1979,2017-04-28,2.399824,78.028807,63.681255,2.399268,77.343056,62.247131,2018-04-30,Receiver,1.072642,-75,120.0,1y,10.0Y


In [94]:
sort  = ff['StrikeRelative'].unique()
sort2 = [0.0 if x == 'ATM' else x for x in sort]
sort2 = list(map(int,sort2))
sort2.sort()
sort2 = list(map(str,sort2))
sort2 = ['ATM' if x == '0' else x for x in sort2]

In [14]:
source = ColumnDataSource(data=dict(x=[],y=[]))

In [15]:
skews = df['StrikeRelative'].unique().tolist()

In [16]:
menu1 = Select(options=df['Exp'].unique().tolist(),value='1y',title='Expiry')
menu2 = Select(options=tails,value='10.0Y',title='Tail')
menu3 = Select(options=skews,value='ATM',title='Strike')
# new menu3 = Select(options=source.data['StrikeRelative'].unique().tolist(),value='ATM',title='Strike')
#menu4 = Select(options=df['OptionType'].unique().tolist(),value='Payer',title='Payer/Receiver')


In [98]:
p = figure(x_axis_type="datetime",plot_width=300, plot_height=300,tools='crosshair,box_select,wheel_zoom,box_zoom,reset')
p.line(x="x",y="y", source=source)


GlyphRenderer(id='11e5daa8-9970-4933-a7e3-3fc825162460', ...)

In [18]:
def select_data():
    selected = df[
        (df["Exp"]==menu1.value)&
        (df["StrikeRelative"]==menu3.value)&
        (df["Tail"]==menu2.value)
    ]
    return selected

In [19]:
def update():
    df2=select_data()
    source.data = dict(
    x=df2['ValueDate'],
    y=df2['ImpliedConsensusVolatility']
    )
    push_notebook()

In [95]:
def update_skew(attr, old, new):
    selected = df[
        (df["Exp"]==menu1.value)&
        (df["Tail"]==menu2.value)
    ]
    sort  = selected['StrikeRelative'].unique()
    sort2 = [0.0 if x == 'ATM' else x for x in sort]
    sort2 = list(map(int,sort2))
    sort2.sort()
    sort2 = list(map(str,sort2))
    sort2 = ['ATM' if x == '0' else x for x in sort2]
    
    menu3.options = sort2
    

In [21]:
menu1.on_change("value",update_skew)
menu2.on_change("value",update_skew)
#menu4.on_change("value",update_skew)

In [22]:
controls = [menu1,menu2,menu3]

In [23]:
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

In [24]:
column1 = column(widgetbox(menu1),widgetbox(menu2),widgetbox(menu3))
layout = row(column1,p)

In [25]:
show(layout,notebook_handle=True)

In [26]:
curdoc().add_root(layout)

In [ ]:
#bokeh serve --show myapp.py